In [33]:
import torch
import torchvision

import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch import nn
from torch import optim
from torch.utils.data import DataLoader

from collections import OrderedDict

# Load Data

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


# Simple CNN

In [6]:
class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        """
        Define the layers of the convolutional neural network.

        Parameters:
            in_channels: int
                The number of channels in the input image. For MNIST, this is 1 (grayscale images).
            num_classes: int
                The number of classes we want to predict, in our case 10 (digits 0 to 9).
        """
        super(CNN, self).__init__()

        
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=3, stride=1, padding=1)
        
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))  # Apply first convolution and ReLU activation
        x = self.pool(x)           # Apply max pooling
        
        x = F.relu(self.conv2(x))  # Apply second convolution and ReLU activation
        x = self.pool(x)           # Apply max pooling
        
        x = x.reshape(x.shape[0], -1)  # Flatten the tensor
        x = self.fc1(x)            # Apply fully connected layer
        return x
    

In [7]:
model = CNN(in_channels=3, num_classes=10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:

def train_model(num_epochs = 8):
    for epoch in range(num_epochs):
        print(f"Epoch [{epoch + 1}/{num_epochs}]")
        running_loss = 0.0

        for data, targets in trainloader:
            optimizer.zero_grad()

            scores = model(data)
            loss = criterion(scores, targets)

            # Backward pass: compute the gradients
            loss.backward()

            # Optimization step: update the model parameters
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch + 1} loss: {running_loss / len(trainloader)}')

train_model()

Epoch [1/8]
Epoch 1 loss: 0.8799849401806932
Epoch [2/8]
Epoch 2 loss: 0.8722213489930039
Epoch [3/8]
Epoch 3 loss: 0.8685711300205392
Epoch [4/8]
Epoch 4 loss: 0.8576208608382193
Epoch [5/8]
Epoch 5 loss: 0.855797382952917
Epoch [6/8]
Epoch 6 loss: 0.8484135303655853
Epoch [7/8]
Epoch 7 loss: 0.8441814763466721
Epoch [8/8]
Epoch 8 loss: 0.8387239220959452


In [13]:

def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():  # Disable gradient calculation
        for x, y in loader:
            # Forward pass: compute the model output
            scores = model(x)
            _, predictions = scores.max(1)  # Get the index of the max log-probability
            num_correct += (predictions == y).sum()  # Count correct predictions
            num_samples += predictions.size(0)  # Count total samples

        # Calculate accuracy
        accuracy = float(num_correct) / float(num_samples) * 100
        print(f"Got {num_correct}/{num_samples} with accuracy {accuracy:.2f}%")
    
    model.train()  # Set the model back to training mode

check_accuracy(trainloader, model)
check_accuracy(testloader, model)

Checking accuracy on training data
Got 36059/50000 with accuracy 72.12%
Checking accuracy on test data
Got 6608/10000 with accuracy 66.08%


In [14]:
model = CNN(in_channels=3, num_classes=10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

train_model()
check_accuracy(trainloader, model)
check_accuracy(testloader, model)


Epoch [1/8]
Epoch 1 loss: 2.1659052379600836
Epoch [2/8]
Epoch 2 loss: 1.8032552433745634
Epoch [3/8]
Epoch 3 loss: 1.5784929987719603
Epoch [4/8]
Epoch 4 loss: 1.4560905600447789
Epoch [5/8]
Epoch 5 loss: 1.375955175103434
Epoch [6/8]
Epoch 6 loss: 1.3183331810452443
Epoch [7/8]
Epoch 7 loss: 1.2723014319644255
Epoch [8/8]
Epoch 8 loss: 1.2341069752145606
Checking accuracy on training data
Got 28268/50000 with accuracy 56.54%
Checking accuracy on test data
Got 5505/10000 with accuracy 55.05%


In [15]:
model = CNN(in_channels=3, num_classes=10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

train_model()
check_accuracy(trainloader, model)
check_accuracy(testloader, model)

Epoch [1/8]
Epoch 1 loss: 1.9939509351235216
Epoch [2/8]
Epoch 2 loss: 1.7466479709081333
Epoch [3/8]
Epoch 3 loss: 1.6263347707136209
Epoch [4/8]
Epoch 4 loss: 1.5375859632211573
Epoch [5/8]
Epoch 5 loss: 1.4859731037293553
Epoch [6/8]
Epoch 6 loss: 1.45196577258732
Epoch [7/8]
Epoch 7 loss: 1.4252829826091562
Epoch [8/8]
Epoch 8 loss: 1.402603363441994
Checking accuracy on training data
Got 25447/50000 with accuracy 50.89%
Checking accuracy on test data
Got 5042/10000 with accuracy 50.42%


In [16]:
model = CNN(in_channels=3, num_classes=10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

train_model()
check_accuracy(trainloader, model)
check_accuracy(testloader, model)

Epoch [1/8]
Epoch 1 loss: 1.5438736866197318
Epoch [2/8]
Epoch 2 loss: 1.3512472524057568
Epoch [3/8]
Epoch 3 loss: 1.311389172168644
Epoch [4/8]
Epoch 4 loss: 1.2967470094675908
Epoch [5/8]
Epoch 5 loss: 1.283280728418199
Epoch [6/8]
Epoch 6 loss: 1.2677870510179368
Epoch [7/8]
Epoch 7 loss: 1.2571021383985534
Epoch [8/8]
Epoch 8 loss: 1.245619918836657
Checking accuracy on training data
Got 29178/50000 with accuracy 58.36%
Checking accuracy on test data
Got 5562/10000 with accuracy 55.62%


In [17]:
model = CNN(in_channels=3, num_classes=10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model()
check_accuracy(trainloader, model)
check_accuracy(testloader, model)

Epoch [1/8]
Epoch 1 loss: 1.574443610885259
Epoch [2/8]
Epoch 2 loss: 1.2894108639196362
Epoch [3/8]
Epoch 3 loss: 1.1752804587869083
Epoch [4/8]
Epoch 4 loss: 1.1114105563944259
Epoch [5/8]
Epoch 5 loss: 1.064007526392217
Epoch [6/8]
Epoch 6 loss: 1.031162334143963
Epoch [7/8]
Epoch 7 loss: 1.0070416435713658
Epoch [8/8]
Epoch 8 loss: 0.9861961019313549
Checking accuracy on training data
Got 33258/50000 with accuracy 66.52%
Checking accuracy on test data
Got 6316/10000 with accuracy 63.16%


# ResNet-18

In [18]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(x)
        out = self.relu(out)
        return out

class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(BasicBlock, 64, 2, stride=1)
        self.layer2 = self._make_layer(BasicBlock, 128, 2, stride=2)
        self.layer3 = self._make_layer(BasicBlock, 256, 2, stride=2)
        self.layer4 = self._make_layer(BasicBlock, 512, 2, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [19]:
model = ResNet18()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model()

Epoch [1/8]
Epoch 1 loss: 1.2214233872225828
Epoch [2/8]
Epoch 2 loss: 0.7871444686065854
Epoch [3/8]
Epoch 3 loss: 0.6001955284486951
Epoch [4/8]
Epoch 4 loss: 0.4724954961205992
Epoch [5/8]
Epoch 5 loss: 0.3634240894061525
Epoch [6/8]
Epoch 6 loss: 0.2653257297471051
Epoch [7/8]
Epoch 7 loss: 0.19090205807324565
Epoch [8/8]
Epoch 8 loss: 0.13389209016819326


In [20]:
check_accuracy(trainloader, model)
check_accuracy(testloader, model)

Checking accuracy on training data
Got 48783/50000 with accuracy 97.57%
Checking accuracy on test data
Got 8118/10000 with accuracy 81.18%


In [ ]:
model = ResNet18()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(4)

In [ ]:
check_accuracy(trainloader, model)
check_accuracy(testloader, model)

## Simplify network

In [23]:
class ResNet18_2(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18_2, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(BasicBlock, 64, 2, stride=1)
        self.layer4 = self._make_layer(BasicBlock, 512, 2, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        
        out = self.layer1(out)
        out = self.layer4(out)
        
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [24]:
model = ResNet18_2()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model()

Epoch [1/8]
Epoch 1 loss: 1.2516906297267856
Epoch [2/8]
Epoch 2 loss: 0.8542207384582066
Epoch [3/8]
Epoch 3 loss: 0.6615574691835266
Epoch [4/8]
Epoch 4 loss: 0.5421851426172439
Epoch [5/8]
Epoch 5 loss: 0.45480854452952096
Epoch [6/8]
Epoch 6 loss: 0.3784521344925284
Epoch [7/8]
Epoch 7 loss: 0.3052793779622411
Epoch [8/8]
Epoch 8 loss: 0.24215169484391236


In [25]:
check_accuracy(trainloader, model)
check_accuracy(testloader, model)

Checking accuracy on training data
Got 46834/50000 with accuracy 93.67%
Checking accuracy on test data
Got 8372/10000 with accuracy 83.72%


In [29]:
class ResNet18_3(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18_3, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer4 = self._make_layer(BasicBlock, 512, 2, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        
        out = self.layer4(out)
        
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [30]:
model = ResNet18_3()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model()

Epoch [1/8]
Epoch 1 loss: 1.2355274414772268
Epoch [2/8]
Epoch 2 loss: 0.8313470598300705
Epoch [3/8]
Epoch 3 loss: 0.6458148317949851
Epoch [4/8]
Epoch 4 loss: 0.522364522566271
Epoch [5/8]
Epoch 5 loss: 0.42518261004515623
Epoch [6/8]
Epoch 6 loss: 0.33481515701050346
Epoch [7/8]
Epoch 7 loss: 0.25957136394460795
Epoch [8/8]
Epoch 8 loss: 0.19074998393921597


In [31]:
check_accuracy(trainloader, model)
check_accuracy(testloader, model)

Checking accuracy on training data
Got 46225/50000 with accuracy 92.45%
Checking accuracy on test data
Got 8024/10000 with accuracy 80.24%


# Pre-trained ResNet-18

In [ ]:
# load ResNet-18
model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)

In [ ]:
model

In [ ]:
print("Original final layer")
print(model.fc)

num_classes = 10
num_ftrs = model.fc.in_features

classifier = nn.Sequential(
    OrderedDict(
        [
            ("fc", nn.Linear(num_ftrs, num_classes)),
            ("output", nn.LogSoftmax(dim = 1)),
        ]
    )
)
model.fc = classifier

print("\nModified final layer")
print(model.fc)

In [ ]:
# Fine-tuing ResNet-18

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(4)

In [ ]:
check_accuracy(trainloader, model)
check_accuracy(testloader, model)

In [ ]:
# No fine-tuning: use ResNet-18 to extract features and train a Logistic Regression

In [35]:
model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)
for parameter in model.parameters():
    parameter.requires_grad = False

num_classes = 10
num_ftrs = model.fc.in_features
    
classifier = nn.Sequential(
    OrderedDict(
        [
            ("fc", nn.Linear(num_ftrs, num_classes)),
            ("output", nn.LogSoftmax(dim = 1)),
        ]
    )
)
model.fc = classifier

In [ ]:
for name, parameter in model.named_parameters():
    print(name, ":\t", parameter.requires_grad)

In [36]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
train_model()

Epoch [1/8]
Epoch 1 loss: 1.726463814830536
Epoch [2/8]
Epoch 2 loss: 1.5996148958230567
Epoch [3/8]
Epoch 3 loss: 1.5855870265180192
Epoch [4/8]
Epoch 4 loss: 1.5703868054977768
Epoch [5/8]
Epoch 5 loss: 1.570842364560003
Epoch [6/8]
Epoch 6 loss: 1.5681217157322427
Epoch [7/8]
Epoch 7 loss: 1.5628956089849058
Epoch [8/8]
Epoch 8 loss: 1.5690741220398632


In [37]:
check_accuracy(trainloader, model)
check_accuracy(testloader, model)

Checking accuracy on training data
Got 23332/50000 with accuracy 46.66%
Checking accuracy on test data
Got 4521/10000 with accuracy 45.21%


# Other pre-trained model

In [ ]:
model = torchvision.models.resnet34(weights=torchvision.models.ResNet34_Weights.DEFAULT)

In [ ]:
num_classes = 10
num_ftrs = model.fc.in_features

classifier = nn.Sequential(
    OrderedDict(
        [
            ("fc", nn.Linear(num_ftrs, num_classes)),
            ("output", nn.LogSoftmax(dim = 1)),
        ]
    )
)
model.fc = classifier

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(4)

check_accuracy(trainloader, model)
check_accuracy(testloader, model)

In [ ]:
model = torchvision.models.mobilenet_v2(weights=torchvision.models.MobileNet_V2_Weights.DEFAULT)

num_classes = 10
num_ftrs = model.classifier[1].in_features

model.classifier[1] = nn.Linear(in_features=num_ftrs, out_features=num_classes, bias=True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(4)

In [ ]:
check_accuracy(trainloader, model)
check_accuracy(testloader, model)